# 🎨 Jack & Sage — Shirt Design Tagger
**Few-shot prompting with Claude Vision API**

This notebook sends each shirt design PNG to Claude, which classifies it using your tagging schema + human-tagged examples as guidance.

### Workflow
1. Install dependencies & configure API key
2. Define schema + few-shot examples
3. Test on a single image
4. Batch process all images
5. Export & validate results

## Cell 1: Install Dependencies

In [ ]:
!pip install openai pandas python-dotenv openpyxl Pillow -q
!pip install python-dotenv

## Cell 2: Imports & Configuration

In [2]:
from openai import OpenAI
import google.generativeai as genai
import base64
import json
import os
import time
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
# ============================================
# ⚙️  CONFIGURE THESE THREE SETTINGS
# ============================================
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)
genai.configure(api_key=GEMINI_API_KEY)
IMAGE_FOLDER = "./artwork_images"          # 📁 Folder with your PNGs
OUTPUT_FILE = "tagged_master_table.csv"    # 💾 Output filename
OPENAI_MODEL = "gpt-5.2"       # 🤖 Model (sonnet = fast + cheap)
GEMINI_MODEL = "gemini-2.5-flash"

print("✅ Setup complete")

C:\Users\smdan\AppData\Local\Temp\ipykernel_44448\766454932.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


✅ Setup complete


## Cell 3: Tagging Schema

In [3]:
TAGGING_SCHEMA = {
    "Theme": [
        "Outdoors", "Mountains", "Ski", "Snow", "Lake Life", "Beach/Coastal",
        "Nautical", "Surf", "Tropics", "Western", "Desert", "Breweries",
        "College/Team", "Patriotic", "Holiday", "Kids", "General Lifestyle", "Other"
    ],
    "Motifs": [
        "Bike", "Skis", "Snowboard", "Wave", "Palm Tree", "Trout",
        "Fish (Generic)", "Bear", "Elk", "Deer", "Moose", "Pine Tree",
        "Cactus", "Sun", "Compass", "Anchor", "Beer Can", "Beer Hop",
        "Football", "Baseball", "Soccer Ball", "Camping", "Tent", "Campfire",
        "Map/Topo", "Badge/Patch", "Logo/Custom Icon", "Text/Lettering",
        "Abstract/Pattern"
    ],
    "Style": [
        "Line Art", "Vintage/Retro", "Geometric", "Minimal", "Bold/Graphic",
        "Watercolor", "Hand-Drawn", "Photoreal", "Cartoon/Playful",
        "Distressed", "Technical/Topo", "Badge/Emblem", "All-Over Pattern", "Other"
    ]
}

print("✅ Schema loaded")
for key, values in TAGGING_SCHEMA.items():
    print(f"   {key}: {len(values)} options")

✅ Schema loaded
   Theme: 18 options
   Motifs: 29 options
   Style: 14 options


## Cell 4: Few-Shot Examples (Human-Tagged Training Data)

In [30]:
FEW_SHOT_EXAMPLES = [
    {"print_id": "Artwork 870", "theme": "Ski", "motifs": "skiing, flowers, boot, Aspen", "style": "retro, vintage"},
    {"print_id": "Artwork 871", "theme": "Western", "motifs": "cowboy, boot, flowers, Winter Park", "style": "folk, vintage"},
    {"print_id": "Artwork 872", "theme": "Outdoors", "motifs": "hiking, camping, flowers, boot", "style": "retro, vintage"},
    {"print_id": "Artwork 873", "theme": "Western", "motifs": "buffalo, Wyoming, cabin, mountains", "style": "line art, folk"},
    {"print_id": "Artwork 874", "theme": "Outdoors", "motifs": "mountain, bird, hawk, eagle, hiking, camping, Glacier National Park", "style": "line art, folk"},
    {"print_id": "Artwork 875", "theme": "Tropics", "motifs": "beach, ocean, surfing, water, Palm Coast", "style": "line art, retro, Mid Century"},
    {"print_id": "Artwork 876", "theme": "Outdoors", "motifs": "flowers, Alaska, frontier", "style": "line art, folk"},
    {"print_id": "Artwork 877", "theme": "Mountains", "motifs": "moose, mountain, Montana", "style": "line art, folk"},
    {"print_id": "Artwork 878", "theme": "Mountains", "motifs": "moose, mountain, Montana", "style": "line art, folk"},
    {"print_id": "Artwork 879", "theme": "Mountains", "motifs": "mountain, pine trees, Whitefish Montana, sun, moon", "style": "line art, folk"},
    {"print_id": "Artwork 880", "theme": "Mountains", "motifs": "pine tree, mountains, Yellowstone National Park", "style": "retro, vintage"},
    {"print_id": "Artwork 881", "theme": "Tropics", "motifs": "palm tree, beach, San Diego", "style": "retro, Mid Century"},
    {"print_id": "Artwork 882", "theme": "Ski", "motifs": "skiing, skiier, mountain, Stowe Vermont", "style": "retro, vintage"},
    {"print_id": "Artwork 883", "theme": "Mountains", "motifs": "bike, moose, mountain, pine tree, Whitefish Montana", "style": "line art, folk, whimsical"},
    {"print_id": "Artwork 884", "theme": "Outdoors", "motifs": "pine tree, big foot, sasquatch, cryptid", "style": "line art, whimsical"},
    {"print_id": "Artwork 885", "theme": "Outdoors", "motifs": "big foot, sasquatch, cryptid", "style": "folk, minimalist"},
    {"print_id": "Artwork 886", "theme": "Lakes & Rivers", "motifs": "fish, fishhook, fishing, trout", "style": "line art"},
    {"print_id": "Artwork 887", "theme": "Nautical", "motifs": "ocean, sun, waves, water, Bermuda", "style": "Ukiyo-e, retro"},
    {"print_id": "Artwork 888", "theme": "Tropics", "motifs": "palm tree, sun, ocean, beach", "style": "retro, minimalist"},
]
print(f"✅ {len(FEW_SHOT_EXAMPLES)} few-shot examples loaded")

✅ 19 few-shot examples loaded


## Cell 5: Build the System Prompt

In [15]:
def build_system_prompt():
    few_shot_text = ""
    for ex in FEW_SHOT_EXAMPLES:
        few_shot_text += f"""\n- Print ID: {ex['print_id']}
  Theme: {ex['theme']}
  Motifs: {ex['motifs']}
  Style: {ex['style']}"""

    return f"""You are an expert visual tagger for a print-on-demand shirt company called Jack & Sage.

Your job: look at a shirt design image and classify it into 3 categories.

## TAGGING SCHEMA (preferred values, but you may add freeform descriptors when needed)

**Theme** (pick ONE primary theme):
{', '.join(TAGGING_SCHEMA['Theme'])}

**Motifs** (list ALL visible motifs: include location names if text is visible):
{', '.join(TAGGING_SCHEMA['Motifs'])}

**Style** (pick 1-3 that apply):
{', '.join(TAGGING_SCHEMA['Style'])}

## TAGGING RULES
1. Theme: Choose exactly ONE primary theme from the schema. Use 'Other' only as last resort.
2. Motifs: List everything visible: animals, objects, text, place names, symbols. Use schema terms where applicable but ADD freeform terms for anything not listed.
3. Style: Pick from schema terms. You may add freeform descriptors like 'folk', 'Mid Century', 'Ukiyo-e' if they fit.
4. If text/lettering is visible, include location or brand names as motifs.

## HUMAN-TAGGED EXAMPLES (learn from these):
{few_shot_text}

## OUTPUT FORMAT
Respond ONLY with valid JSON - no markdown, no explanation:
{{"print_id": "<filename>", "theme": "<single theme>", "motifs": "<comma-separated>", "style": "<comma-separated>"}}
"""

SYSTEM_PROMPT = build_system_prompt()
print("✅ System prompt built")
print(f"   Length: ~{len(SYSTEM_PROMPT)} characters")

✅ System prompt built
   Length: ~2062 characters


## Cell 6: Image Helpers

In [8]:
def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.standard_b64encode(f.read()).decode("utf-8")

def get_media_type(image_path: str) -> str:
    ext = Path(image_path).suffix.lower()
    return {".png": "image/png", ".jpg": "image/jpeg", ".jpeg": "image/jpeg"}.get(ext, "image/png")

def get_image_files(folder: str) -> list:
    valid_ext = {".png", ".jpg", ".jpeg", ".webp"}
    return sorted([f for f in Path(folder).iterdir() if f.suffix.lower() in valid_ext], key=lambda x: x.name)

print("✅ Helper functions ready")

✅ Helper functions ready


## Cell 7: Tag a Single Image (Test First!)

In [20]:
from dis import Instruction


def tag_single_image(image_path: str, fewshot_examples: list, verbose=True) -> dict:
    """Send one image to Claude and get structured tags."""
    image_data = encode_image_to_base64(image_path)
    media_type = get_media_type(image_path)
    filename = Path(image_path).stem

    if verbose:
        print(f"🔍 Tagging: {Path(image_path).name}...", end=" ")

    messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    for ex in fewshot_examples:
        ex_image_b64 = encode_image_to_base64(ex["image_path"])
        ex_media_type = get_media_type(ex["image_path"])

        expected = {
            "print_id": ex["print_id"],
            "theme": ex["theme"],
            "motifs": ex["motifs"],
            "style": ex["style"]
        }

        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": f"data:{ex_media_type};base64,{ex_image_b64}",
                },
                {
                    "type": "input_text",
                    "text": (
                        "Few-shot example. Tag this shirt design and return JSON with keys: "
                        "print_id, theme, motifs, style."
                    ),
                },
            ],
        })

        messages.append({
            "role": "assistant",
            "content": [
                {"type": "output_text", "text": json.dumps(expected)}
            ],
        })

    # Add the target image request
    image_data = encode_image_to_base64(image_path)
    media_type = get_media_type(image_path)
    filename = Path(image_path).stem

    messages.append({
        "role": "user",
        "content": [
            {"type": "input_image", "image_url": f"data:{media_type};base64,{image_data}"},
            {"type": "input_text", "text": f"Now tag this shirt design. Filename: {filename}. Return JSON only."}
        ],
    })

    if verbose:
        print(f"🔍 Tagging with few-shot images: {Path(image_path).name}...")

    response = client.responses.create(
        model=OPENAI_MODEL,
        input=messages,
    )    

    # response = client.responses.create(
    #     model=OPENAI_MODEL,
    #     input=[
    #         {"role": "system", "content": SYSTEM_PROMPT},
    #         {
    #             "role": "user",
    #             "content": [
    #                 {
    #                     "type": "input_image",
    #                     "image_url": f"data:{media_type};base64,{image_data}",
    #                 },
    #                 {
    #                     "type": "input_text",
    #                     "text": f"Tag this shirt design. Filename: {filename}",
    #                 },
    #             ],
    #         },
    #     ],
    # )

    # print(response.output_text)


    raw = response.output[0].content[0].text
    if raw.startswith("```"): raw = raw.split("\n", 1)[1]
    if raw.endswith("```"): raw = raw.rsplit("```", 1)[0]

    try:
        result = json.loads(raw.strip())
    except json.JSONDecodeError:
        print(f"⚠️ JSON parse error")
        result = {"print_id": filename, "theme": "PARSE_ERROR", "motifs": raw[:100], "style": ""}

    if verbose:
        print(f"✅ {result.get('theme')} | {result.get('motifs', '')[:50]}...")
    return result

In [74]:
# ----- TEST WITH ONE IMAGE -----
FEWSHOT_WITH_IMAGES = [
    {
        "image_path": "./artwork_images/artwork_870.png",
        "print_id": "Artwork 870",
        "theme": "Ski",
        "motifs": ["skiing", "flowers", "boot", "Aspen"],
        "style": ["retro", "vintage"],
    },
    {
        "image_path": "./artwork_images/artwork_871.png",
        "print_id": "Artwork 871",
        "theme": "Western",
        "motifs": ["cowboy", "boot", "flowers", "Winter Park"],
        "style": ["folk", "vintage"],
    },
    {
        "image_path": "./artwork_images/artwork_872.png",
        "print_id": "Artwork 872",
        "theme": "Outdoors",
        "motifs": ["hiking", "camping", "flowers", "boot"],
        "style": ["retro", "vintage"],
    },
]
# result = tag_single_image("./artwork_images/Artwork_873.png", FEWSHOT_WITH_IMAGES)
result = tag_single_image_gemini("./artwork_images/Artwork_883.png", FEWSHOT_WITH_IMAGES)
print(json.dumps(result, indent=2))

print("✅ Single tagger ready — uncomment test line above")

🔍 Tagging: Artwork_883.png... ✅ Outdoors | Moose, Bike, Mountains, Pine Tree, Text/Lettering,...
{
  "print_id": "Artwork_883",
  "theme": "Outdoors",
  "motifs": "Moose, Bike, Mountains, Pine Tree, Text/Lettering, Whitefish",
  "style": "Line Art, Hand-Drawn"
}
✅ Single tagger ready — uncomment test line above


## Cell 8: Batch Process All Images

In [65]:
def tag_all_images(folder: str, delay: float = 1.0) -> pd.DataFrame:
    """Process all images in a folder."""
    image_files = get_image_files(folder)
    print(f"📁 Found {len(image_files)} images in {folder}\n")

    results, errors = [], []
    for i, img_path in enumerate(image_files, 1):
        print(f"[{i}/{len(image_files)}] ", end="")
        try:
            results.append(tag_single_image_gemini(str(img_path), FEWSHOT_WITH_IMAGES))
        except Exception as e:
            print(f"❌ {e}")
            errors.append({"file": img_path.name, "error": str(e)})
        if i < len(image_files):
            time.sleep(delay)

    df = pd.DataFrame(results)
    df.columns = [col.replace('_', ' ').title() for col in df.columns]
    print(f"\n✅ Tagged: {len(results)} | ❌ Errors: {len(errors)}")
    return df

# ----- RUN BATCH -----
# Uncomment when ready:
df_results = tag_all_images(IMAGE_FOLDER, delay=1.0)
df_results.head()

print("✅ Batch processor ready — uncomment above when ready")

📁 Found 19 images in ./artwork_images

[1/19] 🔍 Tagging: Artwork_870.png... ✅ Ski | ['skiing', 'flowers', 'boot', 'Aspen', 'Rocky Mountains', 'pine needles', 'Text/Lettering', '1945', 'cnber']...
[2/19] 🔍 Tagging: Artwork_871.png... ✅ Western | cowboy, boot, flowers, Text/Lettering, Winter Park...
[3/19] 🔍 Tagging: Artwork_872.png... ✅ Outdoors | hiking, camping, flowers, boot...
[4/19] 🔍 Tagging: Artwork_873.png... ✅ Western | Bison, Mountains, Pine Tree, Cabin, Flowers, Text/...
[5/19] 🔍 Tagging: Artwork_874.png... ✅ Outdoors | Mountains, Pine Tree, Eagle, River, Clouds, Person...
[6/19] 🔍 Tagging: Artwork_875.png... ✅ Beach/Coastal | Wave, Surfboard, Sun, Clouds, Plants, Birds, Text/...
[7/19] 🔍 Tagging: Artwork_876.png... ✅ Outdoors | Text/Lettering (ALASKA, THE LAST FRONTIER), Flower...
[8/19] 🔍 Tagging: Artwork_877.png... ✅ Outdoors | Moose, Mountains, Sun, River, Trees, Text/Letterin...
[9/19] 🔍 Tagging: Artwork_878.png... ✅ Outdoors | Moose, Mountains, Sun, River, Trees, Plants

## Cell 9: Export Results

In [66]:
def export_results(df, output_path=OUTPUT_FILE):
    df.to_csv(output_path, index=False)
    print(f"💾 CSV saved: {output_path}")

    xlsx = output_path.replace('.csv', '.xlsx')
    df.to_excel(xlsx, index=False, sheet_name='Gemini Master Table')
    print(f"💾 Excel saved: {xlsx}")

    print(f"\n📊 Total tagged: {len(df)}")
    if 'Theme' in df.columns:
        print("\nTheme distribution:")
        print(df['Theme'].value_counts().to_string())

# Uncomment when you have results:
export_results(df_results)

print("✅ Export ready")

💾 CSV saved: tagged_master_table_gemini.csv
💾 Excel saved: tagged_master_table_gemini.xlsx

📊 Total tagged: 19

Theme distribution:
Theme
Outdoors         7
Beach/Coastal    3
Western          2
Mountains        2
ERROR            2
Ski              1
Lake Life        1
Tropics          1
✅ Export ready


In [67]:
import pandas as pd

dfx = pd.read_csv("tagged_master_table_gemini.csv")

dfx

,Print Id,Theme,Motifs,Style
0,Artwork_870,Ski,"['skiing', 'flowers', 'boot', 'Aspen', 'Rocky ...","['retro', 'vintage', 'Line Art']"
1,Artwork_871,Western,"cowboy, boot, flowers, Text/Lettering, Winter ...","Vintage/Retro, Folk"
2,Artwork 872,Outdoors,"hiking, camping, flowers, boot","retro, vintage"
3,Artwork_873,Western,"Bison, Mountains, Pine Tree, Cabin, Flowers, T...","Hand-Drawn, Bold/Graphic, Vintage/Retro"
4,Artwork_874,Outdoors,"Mountains, Pine Tree, Eagle, River, Clouds, Pe...","Line Art, Hand-Drawn, Badge/Emblem"
5,Artwork_875,Beach/Coastal,"Wave, Surfboard, Sun, Clouds, Plants, Birds, T...","Line Art, Vintage/Retro, Bold/Graphic"
6,Artwork_876,Outdoors,"Text/Lettering (ALASKA, THE LAST FRONTIER), Fl...","Line Art, Vintage/Retro, Badge/Emblem"
7,Artwork_877,Outdoors,"Moose, Mountains, Sun, River, Trees, Text/Lett...","Line Art, Hand-Drawn, Vintage"
8,Artwork_878,Outdoors,"Moose, Mountains, Sun, River, Trees, Plants, T...","Line Art, Hand-Drawn, Vintage/Retro"
9,Artwork_879,Mountains,"Pine Tree, Mountains, Sun, Text/Lettering, Whi...","Minimal, Distressed, Bold/Graphic"


## Cell 10: Validate AI vs Human Tags

In [ ]:
def validate_against_human(ai_df, human_examples):
    """Compare AI vs human tags for QA."""
    human_df = pd.DataFrame(human_examples)
    # human_df.columns = [c.replace('_', ' ').title() for c in human_df.columns]
    # human_df.rename(columns={"print_id": "Print Id"}, inplace=True)
    # human_df.rename(columns={"motifs": "Motifs"}, inplace=True)
    # human_df.rename(columns={"style": "Style"}, inplace=True)    
    # print("AI: ", ai_df.columns)
    # print("Human: ", human_df.columns)
    # merged = human_df.merge(ai_df, on='Print Id', suffixes=('_human', '_ai'))
    def norm_cols(df):
        df = df.copy()
        df.columns = (
            df.columns.str.strip()
                     .str.replace("_", " ", regex=False)
                     .str.title()
        )
        return df

    human_df = norm_cols(human_df)
    ai_df = norm_cols(ai_df)

    # Ensure the key exists
    if "Print Id" not in human_df.columns:
        raise KeyError(f"Human DF missing 'Print Id'. Has: {list(human_df.columns)}")
    if "Print Id" not in ai_df.columns:
        raise KeyError(f"AI DF missing 'Print Id'. Has: {list(ai_df.columns)}")

    # Normalize join keys (avoid whitespace/case mismatch)
    human_df["Print Id"] = human_df["Print Id"].astype(str).str.strip()
    ai_df["Print Id"] = ai_df["Print Id"].astype(str).str.strip()

    merged = human_df.merge(ai_df, on="Print Id", suffixes=("_human", "_ai"))
    print("Merged: ", merged)
    comp = []
    for _, r in merged.iterrows():
        comp.append({
            'Print ID': r['Print Id'],
            'Theme Match': r.get('Theme_human','') == r.get('Theme_ai',''),
            'Human Theme': r.get('Theme_human',''),
            'AI Theme': r.get('Theme_ai',''),
            'Human Motifs': r.get('Motifs_human',''),
            'AI Motifs': r.get('Motifs_ai',''),
        })

    comp_df = pd.DataFrame(comp)
    match_rate = comp_df['Theme Match'].mean() * 100
    print(f"🎯 Theme match rate: {match_rate:.1f}%")
    return comp_df

# Uncomment after batch:
comparison = validate_against_human(df_results, FEW_SHOT_EXAMPLES)
# comparison

print("✅ Validator ready")
print()
print("🚀 READY! Steps:")
print("  1. Set API key in Cell 2")
print("  2. Put PNGs in ./artwork_images/")
print("  3. Test one image (Cell 7)")
print("  4. Batch all images (Cell 8)")
print("  5. Export (Cell 9)")
print("  6. Validate (Cell 10)")

Merged:  Empty DataFrame
Columns: [Print Id, Theme_human, Motifs_human, Style_human, Theme_ai, Motifs_ai, Style_ai]
Index: []


KeyError: 'Theme Match'

In [69]:
def validate_against_human(ai_df, human_examples):
    human_df = pd.DataFrame(human_examples)

    # Normalize column names
    human_df.columns = human_df.columns.str.strip().str.replace("_", " ", regex=False).str.title()
    ai_df = ai_df.copy()
    ai_df.columns = ai_df.columns.str.strip().str.replace("_", " ", regex=False).str.title()

    # Ensure join key exists
    if "Print Id" not in human_df.columns:
        raise KeyError(f"Human missing 'Print Id'. Has: {list(human_df.columns)}")
    if "Print Id" not in ai_df.columns:
        raise KeyError(f"AI missing 'Print Id'. Has: {list(ai_df.columns)}")

    # Normalize Print Id values so Artwork_870 == Artwork 870
    def norm_id(s):
        return str(s).strip().lower().replace("_", " ")

    human_df["Print Id"] = human_df["Print Id"].map(norm_id)
    ai_df["Print Id"] = ai_df["Print Id"].map(norm_id)

    merged = human_df.merge(ai_df, on="Print Id", suffixes=("_human", "_ai"))

    # If nothing matched, return an empty comparison safely
    if merged.empty:
        print("⚠️ No matches found in merge (Print Id values don’t align).")
        print("Human sample:", human_df["Print Id"].head(5).tolist())
        print("AI sample   :", ai_df["Print Id"].head(5).tolist())
        return pd.DataFrame(columns=["Print Id", "Theme Match", "Human Theme", "AI Theme", "Human Motifs", "AI Motifs"])

    comp = []
    for _, r in merged.iterrows():
        comp.append({
            "Print Id": r["Print Id"],
            "Theme Match": r.get("Theme_human", "") == r.get("Theme_ai", ""),
            "Human Theme": r.get("Theme_human", ""),
            "AI Theme": r.get("Theme_ai", ""),
            "Human Motifs": r.get("Motifs_human", ""),
            "AI Motifs": r.get("Motifs_ai", ""),
        })

    comp_df = pd.DataFrame(comp)
    match_rate = comp_df["Theme Match"].mean() * 100
    print(f"🎯 Theme match rate: {match_rate:.1f}%")
    return comp_df

df_results_gemini = pd.read_csv("tagged_master_table_gemini.csv")
comparison = validate_against_human(df_results_gemini, FEW_SHOT_EXAMPLES)
comparison


🎯 Theme match rate: 57.9%


,Print Id,Theme Match,Human Theme,AI Theme,Human Motifs,AI Motifs
0,artwork 870,True,Ski,Ski,"skiing, flowers, boot, Aspen","['skiing', 'flowers', 'boot', 'Aspen', 'Rocky ..."
1,artwork 871,True,Western,Western,"cowboy, boot, flowers, Winter Park","cowboy, boot, flowers, Text/Lettering, Winter ..."
2,artwork 872,True,Outdoors,Outdoors,"hiking, camping, flowers, boot","hiking, camping, flowers, boot"
3,artwork 873,True,Western,Western,"buffalo, Wyoming, cabin, mountains","Bison, Mountains, Pine Tree, Cabin, Flowers, T..."
4,artwork 874,True,Outdoors,Outdoors,"mountain, bird, hawk, eagle, hiking, camping, ...","Mountains, Pine Tree, Eagle, River, Clouds, Pe..."
5,artwork 875,False,Tropics,Beach/Coastal,"beach, ocean, surfing, water, Palm Coast","Wave, Surfboard, Sun, Clouds, Plants, Birds, T..."
6,artwork 876,True,Outdoors,Outdoors,"flowers, Alaska, frontier","Text/Lettering (ALASKA, THE LAST FRONTIER), Fl..."
7,artwork 877,False,Mountains,Outdoors,"moose, mountain, Montana","Moose, Mountains, Sun, River, Trees, Text/Lett..."
8,artwork 878,False,Mountains,Outdoors,"moose, mountain, Montana","Moose, Mountains, Sun, River, Trees, Plants, T..."
9,artwork 879,True,Mountains,Mountains,"mountain, pine trees, Whitefish Montana, sun, ...","Pine Tree, Mountains, Sun, Text/Lettering, Whi..."


In [62]:
comparison.to_csv("OPENAI_Comparison_with_Human_Tagging.csv")

In [63]:
import google.generativeai as genai
import json
import os
import time
import pandas as pd
from pathlib import Path
from PIL import Image
from dotenv import load_dotenv

load_dotenv()

# ============================================
# ⚙️ GEMINI CONFIGURATION
# ============================================
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

IMAGE_FOLDER = "./artwork_images"
OUTPUT_FILE = "tagged_master_table_gemini.csv"
GEMINI_MODEL = "gemini-2.5-flash"  # Highly recommended for speed/cost

model = genai.GenerativeModel(model_name=GEMINI_MODEL)
print("✅ Gemini Setup complete")

✅ Gemini Setup complete


In [64]:

def tag_single_image_gemini(image_path: str, fewshot_examples: list, verbose=True) -> dict:
    """Send one image to Gemini with few-shot examples and get structured tags."""
    target_img = Image.open(image_path)
    filename = Path(image_path).stem

    if verbose:
        print(f"🔍 Tagging: {Path(image_path).name}...", end=" ")

    # Build the multimodal prompt sequence
    prompt_parts = [SYSTEM_PROMPT]

    # Add few-shot examples (Image + Description)
    for ex in fewshot_examples:
        prompt_parts.append(Image.open(ex["image_path"]))
        prompt_parts.append(f"Example Tagging for {ex['print_id']}: {json.dumps(ex)}")

    # Add the target image
    prompt_parts.append(target_img)
    prompt_parts.append(f"Now tag this shirt design. Filename: {filename}. Return JSON only.")

    try:
        # Use generation_config to enforce JSON response if using Pro/Flash models
        response = model.generate_content(
            prompt_parts,
            generation_config={"response_mime_type": "application/json"}
        )
        
        raw = response.text
        result = json.loads(raw.strip())
    except Exception as e:
        if verbose: print(f"⚠️ Error: {str(e)}")
        result = {"print_id": filename, "theme": "ERROR", "motifs": str(e), "style": ""}

    if verbose:
        print(f"✅ {result.get('theme')} | {result.get('motifs', '')[:50]}...")
    
    return result

In [1]:
# JUPYTER NOTEBOOK CELL 1
# Pure-Python PNG XMP writer (no exiftool, no extra libs).
# Writes/updates an XMP packet inside a PNG iTXt chunk with keyword: "XML:com.adobe.xmp"
#
# Works by:
# - Reading PNG chunks
# - Removing existing XMP iTXt chunks (if any)
# - Inserting a new XMP iTXt chunk (before IEND)
#
# Output: creates a NEW PNG (keeps original untouched)

import os
import struct
import binascii
from datetime import datetime
from xml.sax.saxutils import escape

PNG_SIGNATURE = b"\x89PNG\r\n\x1a\n"
XMP_KEYWORD = b"XML:com.adobe.xmp"

def _crc(chunk_type: bytes, data: bytes) -> int:
    return binascii.crc32(chunk_type + data) & 0xffffffff

def _pack_chunk(chunk_type: bytes, data: bytes) -> bytes:
    return struct.pack(">I", len(data)) + chunk_type + data + struct.pack(">I", _crc(chunk_type, data))

def _read_chunks(png_bytes: bytes):
    if not png_bytes.startswith(PNG_SIGNATURE):
        raise ValueError("Not a valid PNG (bad signature).")
    i = len(PNG_SIGNATURE)
    chunks = []
    while i < len(png_bytes):
        if i + 8 > len(png_bytes):
            raise ValueError("Corrupt PNG (unexpected EOF while reading chunk header).")
        length = struct.unpack(">I", png_bytes[i:i+4])[0]
        ctype = png_bytes[i+4:i+8]
        i += 8
        if i + length + 4 > len(png_bytes):
            raise ValueError("Corrupt PNG (unexpected EOF while reading chunk data).")
        data = png_bytes[i:i+length]
        i += length
        crc = struct.unpack(">I", png_bytes[i:i+4])[0]
        i += 4
        # (Optional) CRC check could be done here.
        chunks.append((ctype, data, crc))
        if ctype == b"IEND":
            break
    return chunks

def _is_xmp_itxt_chunk(chunk_type: bytes, data: bytes) -> bool:
    if chunk_type != b"iTXt":
        return False
    # iTXt layout:
    # keyword (null-terminated)
    # compression_flag (1 byte)
    # compression_method (1 byte)
    # language_tag (null-terminated)
    # translated_keyword (null-terminated)
    # text (rest)
    nul = data.find(b"\x00")
    if nul == -1:
        return False
    keyword = data[:nul]
    return keyword == XMP_KEYWORD

def _build_itxt_data(keyword: bytes, text_utf8: bytes) -> bytes:
    # Uncompressed iTXt:
    # keyword + \0
    # compression_flag=0
    # compression_method=0
    # language_tag + \0  (empty)
    # translated_keyword + \0 (empty)
    # text
    if b"\x00" in keyword:
        raise ValueError("Keyword may not contain null bytes.")
    return (
        keyword + b"\x00" +
        b"\x00" +          # compression flag (0 = uncompressed)
        b"\x00" +          # compression method
        b"\x00" +          # language tag terminator (empty string)
        b"\x00" +          # translated keyword terminator (empty string)
        text_utf8
    )

def _xmp_bag(items):
    # items: list[str]
    # Creates <rdf:Bag><rdf:li>...</rdf:li>...</rdf:Bag>
    lis = "\n".join([f'      <rdf:li>{escape(str(x))}</rdf:li>' for x in items if str(x).strip()])
    return f"<rdf:Bag>\n{lis}\n    </rdf:Bag>"

def build_xmp_packet(
    tags: dict,
    title: str | None = None,
    creator_tool: str = "jupyter-xmp-writer",
    include_dc_subject_from_all: bool = True,
) -> str:
    """
    tags example:
    {
      "theme": ["ski", "mountain"],
      "motifs": ["trees", "lake"],
      "style": ["vintage"],
      "tone": ["outdoor"],
      "primary_color": ["blue"]
    }
    """
    # Normalize to list[str]
    norm = {}
    for k, v in tags.items():
        if v is None:
            norm[k] = []
        elif isinstance(v, (list, tuple, set)):
            norm[k] = [str(x) for x in v]
        else:
            norm[k] = [str(v)]

    # Build a single keyword list for broad compatibility (dc:subject)
    all_keywords = []
    if include_dc_subject_from_all:
        for key in ["theme", "motifs", "style", "tone", "primary_color"]:
            all_keywords.extend(norm.get(key, []))
        # Deduplicate while preserving order
        seen = set()
        all_keywords = [x for x in all_keywords if not (x in seen or seen.add(x))]

    now = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

    # Custom namespace for your structured tags (you can rename it)
    # This lets you keep separate fields AND also have dc:subject for general search.
    xmp = f"""<?xpacket begin="﻿" id="W5M0MpCehiHzreSzNTczkc9d"?>
<x:xmpmeta xmlns:x="adobe:ns:meta/">
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
           xmlns:dc="http://purl.org/dc/elements/1.1/"
           xmlns:xmp="http://ns.adobe.com/xap/1.0/"
           xmlns:photoshop="http://ns.adobe.com/photoshop/1.0/"
           xmlns:aiTags="https://example.com/ai-tags/1.0/">

    <rdf:Description rdf:about=""
      xmp:CreatorTool="{escape(creator_tool)}"
      xmp:CreateDate="{escape(now)}"
      xmp:ModifyDate="{escape(now)}">

      {f"<dc:title><rdf:Alt><rdf:li xml:lang='x-default'>{escape(title)}</rdf:li></rdf:Alt></dc:title>" if title else ""}

      <!-- Broad keywords (most tools read this) -->
      <dc:subject>
        {_xmp_bag(all_keywords)}
      </dc:subject>

      <!-- Photoshop-style keyword field some Adobe apps also recognize -->
      <photoshop:Keywords>{escape(", ".join(all_keywords))}</photoshop:Keywords>

      <!-- Structured fields for your pipeline -->
      <aiTags:theme>
        {_xmp_bag(norm.get("theme", []))}
      </aiTags:theme>

      <aiTags:motifs>
        {_xmp_bag(norm.get("motifs", []))}
      </aiTags:motifs>

      <aiTags:style>
        {_xmp_bag(norm.get("style", []))}
      </aiTags:style>

      <aiTags:tone>
        {_xmp_bag(norm.get("tone", []))}
      </aiTags:tone>

      <aiTags:primary_color>
        {_xmp_bag(norm.get("primary_color", []))}
      </aiTags:primary_color>

    </rdf:Description>
  </rdf:RDF>
</x:xmpmeta>
<?xpacket end="w"?>"""
    return xmp

def write_xmp_to_png(input_png_path: str, output_png_path: str, xmp_packet: str, remove_existing_xmp: bool = True):
    with open(input_png_path, "rb") as f:
        png_bytes = f.read()

    chunks = _read_chunks(png_bytes)

    new_chunks = []
    for ctype, data, crc in chunks:
        if remove_existing_xmp and _is_xmp_itxt_chunk(ctype, data):
            # skip old XMP
            continue
        new_chunks.append((ctype, data))

    # Create new iTXt chunk for XMP
    xmp_bytes = xmp_packet.encode("utf-8")
    itxt_data = _build_itxt_data(XMP_KEYWORD, xmp_bytes)
    itxt_chunk = _pack_chunk(b"iTXt", itxt_data)

    # Rebuild PNG: signature + chunks, inserting XMP before IEND
    out = bytearray()
    out += PNG_SIGNATURE
    inserted = False

    for ctype, data in new_chunks:
        if ctype == b"IEND" and not inserted:
            out += itxt_chunk
            inserted = True
        out += _pack_chunk(ctype, data)

    if not inserted:
        raise ValueError("Invalid PNG: IEND chunk not found.")

    os.makedirs(os.path.dirname(output_png_path) or ".", exist_ok=True)
    with open(output_png_path, "wb") as f:
        f.write(out)

def read_xmp_from_png(png_path: str) -> str | None:
    with open(png_path, "rb") as f:
        png_bytes = f.read()
    chunks = _read_chunks(png_bytes)
    for ctype, data, crc in chunks:
        if _is_xmp_itxt_chunk(ctype, data):
            # Parse iTXt fields
            # keyword\0 flag method lang\0 trans\0 text
            # We'll locate the 5th null separators after the keyword.
            # After keyword\0 (already found), there are 2 bytes, then lang\0 then trans\0 then text
            nul = data.find(b"\x00")
            payload = data[nul+1:]
            if len(payload) < 2:
                return None
            # skip compression flag & method
            payload = payload[2:]
            # language tag (null-terminated)
            nul2 = payload.find(b"\x00")
            if nul2 == -1:
                return None
            payload = payload[nul2+1:]
            # translated keyword (null-terminated)
            nul3 = payload.find(b"\x00")
            if nul3 == -1:
                return None
            text = payload[nul3+1:]
            try:
                return text.decode("utf-8", errors="replace")
            except Exception:
                return None
    return None


In [2]:
# JUPYTER NOTEBOOK CELL 2
# INPUTS: image path + your tags

input_png = "artwork_images/Artwork_882.png"          # <-- set your input file path
output_png = "output_Artwork_882_tagged.png" # <-- output file

tags = {
    "theme": ["Ski"],
    "motifs": ["skiing", "skiier", "mountain", "Stowe Vermont"],
    "style": ["retro", "vintage"],
}

xmp = build_xmp_packet(tags=tags, title="CT1201")

write_xmp_to_png(input_png, output_png, xmp_packet=xmp, remove_existing_xmp=True)

print("Wrote XMP into:", output_png)


Wrote XMP into: output_Artwork_882_tagged.png


In [6]:
# JUPYTER NOTEBOOK CELL 3
# Verify: read back XMP
input_png = "artwork_images/SW1200.png"

xmp_read = read_xmp_from_png(input_png)
print(xmp_read[:800] if xmp_read else "No XMP found")


<?xpacket begin="﻿" id="W5M0MpCehiHzreSzNTczkc9d"?>
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 9.1-c003 79.9690a87, 2025/03/06-19:12:03        ">
   <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
      <rdf:Description rdf:about=""
            xmlns:dc="http://purl.org/dc/elements/1.1/"
            xmlns:xmp="http://ns.adobe.com/xap/1.0/"
            xmlns:xmpGImg="http://ns.adobe.com/xap/1.0/g/img/"
            xmlns:aiThumbnail="http://ns.adobe/meta/ai/thumbnail"
            xmlns:xmpMM="http://ns.adobe.com/xap/1.0/mm/"
            xmlns:stRef="http://ns.adobe.com/xap/1.0/sType/ResourceRef#"
            xmlns:stEvt="http://ns.adobe.com/xap/1.0/sType/ResourceEvent#"
            xmlns:illustrator="http://ns.adobe.com/illustrator/1.0/"
            xmlns:x


In [7]:
print(xmp_read)

<?xpacket begin="﻿" id="W5M0MpCehiHzreSzNTczkc9d"?>
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 9.1-c003 79.9690a87, 2025/03/06-19:12:03        ">
   <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
      <rdf:Description rdf:about=""
            xmlns:dc="http://purl.org/dc/elements/1.1/"
            xmlns:xmp="http://ns.adobe.com/xap/1.0/"
            xmlns:xmpGImg="http://ns.adobe.com/xap/1.0/g/img/"
            xmlns:aiThumbnail="http://ns.adobe/meta/ai/thumbnail"
            xmlns:xmpMM="http://ns.adobe.com/xap/1.0/mm/"
            xmlns:stRef="http://ns.adobe.com/xap/1.0/sType/ResourceRef#"
            xmlns:stEvt="http://ns.adobe.com/xap/1.0/sType/ResourceEvent#"
            xmlns:illustrator="http://ns.adobe.com/illustrator/1.0/"
            xmlns:xmpTPg="http://ns.adobe.com/xap/1.0/t/pg/"
            xmlns:stDim="http://ns.adobe.com/xap/1.0/sType/Dimensions#"
            xmlns:xmpG="http://ns.adobe.com/xap/1.0/g/"
            xmlns:pdf="http://